In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import scanpy as sc
import sklearn
from scipy.stats import pearsonr

In [ ]:
RNA_data = sc.read_h5ad('../scButterfly/data/lymphoma/lymphoma_RNA.h5ad')
RNA_data
genes = list(RNA_data.var_names)
RNA_df = pd.DataFrame(RNA_data.X.toarray(), columns=genes)

/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [ ]:
trrust = pd.read_csv("../Benchmarking/data/trrust_rawdata.human.tsv", sep="\t", header=None)
trrust.columns = ["TF", "Target", "Regulation", "PMID"]
tftg = trrust[["TF", "Target"]].drop_duplicates()


In [ ]:
trrust

,TF,Target,Regulation,PMID
0,AATF,BAX,Repression,22909821
1,AATF,CDKN1A,Unknown,17157788
2,AATF,KLK3,Unknown,23146908
3,AATF,MYC,Activation,20549547
4,AATF,TP53,Unknown,17157788
...,...,...,...,...
9391,ZNF76,CDKN1A,Repression,15280358
9392,ZNF76,PCYT1A,Activation,14702349
9393,ZNF76,TALDO1,Unknown,14702349
9394,ZNRD1,ABCB1,Activation,16373708


In [ ]:
tftg_filtered = tftg[tftg["TF"].isin(genes) & tftg["Target"].isin(genes)].reset_index(drop=True)

In [ ]:
tftg_filtered

,TF,Target
0,AATF,BAX
1,AATF,CDKN1A
2,AATF,KLK3
3,AATF,MYC
4,AATF,TP53
...,...,...
8343,ZNF76,CDKN1A
8344,ZNF76,PCYT1A
8345,ZNF76,TALDO1
8346,ZNRD1,ABCB1


In [ ]:
def compute_tf_target_corr(pairs, expr_df, method="pearson"):
    correlations = []
    for _, row in pairs.iterrows():
        tf_expr = expr_df[row["TF"]]
        target_expr = expr_df[row["Target"]]
        if method == "pearson":
            r_value, p_value = pearsonr(tf_expr, target_expr)
        correlations.append((r_value, p_value))
    return pd.concat([pairs, pd.DataFrame(correlations, columns=["Correlation", "P-value"])], axis=1)

In [ ]:
corr = compute_tf_target_corr(tftg_filtered, RNA_df, method="pearson")

/usr/local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [ ]:
corr

,TF,Target,Correlation,P-value
0,AATF,BAX,0.064453,5.839254e-15
1,AATF,CDKN1A,0.017179,3.762473e-02
2,AATF,KLK3,0.007489,3.648109e-01
3,AATF,MYC,0.063337,1.695127e-14
4,AATF,TP53,0.084531,1.218585e-24
...,...,...,...,...
8343,ZNF76,CDKN1A,0.010632,1.982477e-01
8344,ZNF76,PCYT1A,0.100810,2.154979e-34
8345,ZNF76,TALDO1,0.048268,5.093731e-09
8346,ZNRD1,ABCB1,0.045747,3.049790e-08
